In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [ ]:
class SizeDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.directory, self.files[idx])
        data = np.loadtxt(file_path).flatten()
        return torch.tensor(data, dtype=torch.float32)

human_dataset = SizeDataset("/content/human_sizes")
manga_dataset = SizeDataset("/content/manga_sizes")

human_dataloader = DataLoader(human_dataset, batch_size=5, shuffle=True)
manga_dataloader = DataLoader(manga_dataset, batch_size=5, shuffle=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 48),
            nn.ReLU(),
            nn.Linear(48, 96),
            nn.ReLU(),
            nn.Linear(96, 48),
            nn.ReLU(),
            nn.Linear(48, 4)
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 48),
            nn.ReLU(),
            nn.Linear(48, 24),
            nn.ReLU(),
            nn.Linear(24, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
G_H2M = Generator()
G_M2H = Generator()
D_H = Discriminator()
D_M = Discriminator()

In [ ]:
# Define loss function and optimizers
criterion_GAN = nn.BCELoss()
criterion_cycle = nn.L1Loss()
lr = 0.0002

optimizer_G_H2M = optim.Adam(G_H2M.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_G_M2H = optim.Adam(G_M2H.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_H = optim.Adam(D_H.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_M = optim.Adam(D_M.parameters(), lr=lr, betas=(0.5, 0.999))

# Labels for real and fake data
real_label = torch.ones((5, 1))
fake_label = torch.zeros((5, 1))

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for (human_sizes, manga_sizes) in zip(human_dataloader, manga_dataloader):

        # Train Generators
        optimizer_G_H2M.zero_grad()
        optimizer_G_M2H.zero_grad()

        # GAN loss
        fake_manga = G_H2M(human_sizes)
        pred_fake = D_M(fake_manga)
        loss_GAN_H2M = criterion_GAN(pred_fake, real_label)

        fake_human = G_M2H(manga_sizes)
        pred_fake = D_H(fake_human)
        loss_GAN_M2H = criterion_GAN(pred_fake, real_label)

        # Cycle consistency loss
        recovered_human = G_M2H(fake_manga)
        loss_cycle_H2M = criterion_cycle(recovered_human, human_sizes)

        recovered_manga = G_H2M(fake_human)
        loss_cycle_M2H = criterion_cycle(recovered_manga, manga_sizes)

        # Total generator loss
        loss_G = loss_GAN_H2M + loss_GAN_M2H + 10.0 * (loss_cycle_H2M + loss_cycle_M2H)
        loss_G.backward()

        optimizer_G_H2M.step()
        optimizer_G_M2H.step()

        # Train Discriminator H
        optimizer_D_H.zero_grad()

        # Real loss
        pred_real = D_H(human_sizes)
        loss_D_real = criterion_GAN(pred_real, real_label)

        # Fake loss
        pred_fake = D_H(fake_human.detach())
        loss_D_fake = criterion_GAN(pred_fake, fake_label)

        # Total loss
        loss_D_H = (loss_D_real + loss_D_fake) * 0.5
        loss_D_H.backward()
        optimizer_D_H.step()

        # Train Discriminator M
        optimizer_D_M.zero_grad()

        # Real loss
        pred_real = D_M(manga_sizes)
        loss_D_real = criterion_GAN(pred_real, real_label)

        # Fake loss
        pred_fake = D_M(fake_manga.detach())
        loss_D_fake = criterion_GAN(pred_fake, fake_label)

        # Total loss
        loss_D_M = (loss_D_real + loss_D_fake) * 0.5
        loss_D_M.backward()
        optimizer_D_M.step()

    print(f'Epoch [{epoch+1}/{num_epochs}] Loss G: {loss_G.item()} Loss D_H: {loss_D_H.item()} Loss D_M: {loss_D_M.item()}')

Epoch [1/500] Loss G: 4.3050432205200195 Loss D_H: 0.6955143213272095 Loss D_M: 0.690280556678772
Epoch [2/500] Loss G: 3.6487743854522705 Loss D_H: 0.6940141916275024 Loss D_M: 0.6901440024375916
Epoch [3/500] Loss G: 2.65667986869812 Loss D_H: 0.6927852630615234 Loss D_M: 0.6914514303207397
Epoch [4/500] Loss G: 2.206876754760742 Loss D_H: 0.6925321817398071 Loss D_M: 0.6925336122512817
Epoch [5/500] Loss G: 1.7412374019622803 Loss D_H: 0.6933138966560364 Loss D_M: 0.6932134628295898
Epoch [6/500] Loss G: 1.7538437843322754 Loss D_H: 0.6936435699462891 Loss D_M: 0.6933724880218506
Epoch [7/500] Loss G: 1.824974536895752 Loss D_H: 0.6934335231781006 Loss D_M: 0.692913830280304
Epoch [8/500] Loss G: 1.7642637491226196 Loss D_H: 0.6931970119476318 Loss D_M: 0.6924493312835693
Epoch [9/500] Loss G: 1.7392487525939941 Loss D_H: 0.6932030916213989 Loss D_M: 0.6926558017730713
Epoch [10/500] Loss G: 1.76816725730896 Loss D_H: 0.6930494904518127 Loss D_M: 0.6933069229125977
Epoch [11/500] Lo

In [ ]:
torch.save(G_H2M.state_dict(), 'G_H2M.pth')
torch.save(G_M2H.state_dict(), 'G_M2H.pth')
torch.save(D_H.state_dict(), 'D_H.pth')
torch.save(D_M.state_dict(), 'D_M.pth')

In [ ]:
test_human_dataset = SizeDataset("/content/test_sizes")
test_human_dataloader = DataLoader(test_human_dataset, batch_size=1, shuffle=False)
G_H2M.load_state_dict(torch.load('G_H2M.pth'))
G_H2M.eval()
with torch.no_grad():
    for human_sizes in test_human_dataloader:
        generated_manga_sizes = G_H2M(human_sizes)
        print("generated manga sizes:", generated_manga_sizes)

generated manga sizes: tensor([[0.1591, 0.1532, 0.0869, 0.0911]])
